<a href="https://colab.research.google.com/github/yakaralar/IE492/blob/master/Decision_Trees_LGB_Keras.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
import os
import glob
import shutil
os.listdir()

In [0]:
os.listdir("drive/My Drive/colab_datas/IE 492")

In [0]:
shutil.copy("drive/My Drive/colab_datas/IE 492/RealTimeConsumption_unholidayized.csv","./")
shutil.copy("drive/My Drive/colab_datas/IE 492/updated_support-date_based_information.r","./")
shutil.copy("drive/My Drive/colab_datas/IE 492/RealTimeConsumption-01122014-09032020.csv","./")
shutil.copy("drive/My Drive/colab_datas/IE 492/RealTimeConsumption_holiday.csv","./")

In [0]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import pandas_profiling
%matplotlib inline
from tqdm import tqdm_notebook as tqdm

# Evaluation function

In [120]:
!pip install graphviz
!pip install sklearn --upgrade
from sklearn.metrics import mean_squared_error,mean_squared_log_error,mean_absolute_error
from sklearn.tree import DecisionTreeRegressor,plot_tree

def mean_absolute_percentage_error(y_true, y_pred): 
    return np.mean(np.abs((y_true - y_pred) / y_true)) * 100

eval_df=[]
def eval_function(name_,y_true, y_pred,valid_df):
    """
    docstring denir buraya func paramlarının neler oldugunu yazarsınız
    name_:kullanılan modelin ismi
    y_true:actual values
    y_pred:predicted values
    valid_df:modele verdiginiz sekliyle validation dataframe'i ve yanına datetime'ı
    """
    rmse_=np.sqrt(mean_squared_error(y_true,y_pred))
    rmsle_=np.sqrt(mean_squared_log_error(y_true,y_pred))
    mae_=mean_absolute_error(y_true,y_pred)
    mape_=mean_absolute_percentage_error(y_true, y_pred)

    print("RMSE: ",rmse_)
    print("RMSLE: ",rmsle_)
    print("MAE: ",mae_)
    print("MAPE: ",mape_)
    sns.scatterplot(x=y_pred,y=y_true)
    plt.pause(0.01)
    sns.residplot(y_true,y_pred)
    plt.pause(0.01)
    nchunks=6 # plot count
    fig,ax=plt.subplots(nrows=nchunks,figsize=(32,25))
    valid_copy=valid_df.reset_index(drop=True).copy()
    for i,datetime_ in enumerate(np.array_split(valid_copy["Datetime"],nchunks)):
        sns.lineplot(y=y_true[datetime_.index.tolist()],x=datetime_,color="green",ax=ax[i])
        sns.lineplot(y=y_pred[datetime_.index.tolist()],x=datetime_,color="red",ax=ax[i])
    plt.pause(0.01)
    res_model=DecisionTreeRegressor(max_depth=4)
    _, ax = plt.subplots(figsize=(60, 60))  # whatever size you want
    fig=res_model.fit(valid_df.drop(columns=["Datetime"]),y_true-y_pred)
    plot_tree(fig,ax=ax,feature_names=valid_df.drop(columns=["Datetime"]).columns,fontsize=16)
    plt.pause(0.01)

    ## saatlik mappep tanımla
    return name_, rmse_,rmsle_,mae_,mape_

Requirement already up-to-date: sklearn in /usr/local/lib/python3.6/dist-packages (0.0)


# Consumption df

In [40]:
# bruuuuuuuh
cons_unholidayized=pd.read_csv("RealTimeConsumption_unholidayized.csv",encoding='latin-1')
cons_unholidayized.columns=["Date","Hour","Consumption"]+cons_unholidayized.columns.tolist()[3:]
cons_unholidayized["Datetime"]=cons_unholidayized["Date"]+" "+cons_unholidayized["Hour"]
cons_unholidayized["Datetime"]=pd.to_datetime(cons_unholidayized["Datetime"],format="%d.%m.%Y %H:%M")
cons_unholidayized.head(5)

,Date,Hour,Consumption,Datetime,is_annelergunu,is_arife,is_babalargunu,is_cocukbayrami,is_cumhuriyet,is_elections,is_examination,is_genclikbayrami,is_holiday,is_iscibayrami,is_kadinlargunu,is_kandil,is_kurbanbayrami,is_kurbanbayramilkgun,is_kurbanoncesigunler,is_onbestemmuz,is_outlierdays,is_ramazanbayrami,is_ramazanbayramilkgun,is_ramazanbayramisonrasi,is_ramazangunler,is_ramazanilkgun,is_ramazanoncesigunler,is_school_days,is_school_end,is_school_start,is_semester,is_semester_days,is_sevgililergunu,is_single_day_holidays,is_summerTimes,is_summer_break,is_uzunbayram,is_winterTimes,is_winter_break,is_winter_break_schools,is_yilbasi,is_yilbasioncesi,is_zaferbayrami
0,31.12.2015,00:00,29745.882083,2015-12-31 00:00:00,-0.0,0.0,0.0,0.0,0.0,-0.0,-0.0,0.0,-0.0,0.0,-0.0,-0.0,-0.0,-0.0,-0.0,0.0,0.0,-0.0,-0.0,0.0,-0.0,0.0,-0.0,-193.25695,0.0,0.0,-0.0,0.0,0.0,0.0,-0.0,0.0,-0.0,-0.0,-0.0,0.0,-0.0,38.254867,0.0
1,31.12.2015,01:00,27940.432083,2015-12-31 01:00:00,-0.0,0.0,0.0,0.0,0.0,-0.0,-0.0,0.0,-0.0,0.0,-0.0,-0.0,-0.0,-0.0,-0.0,0.0,0.0,-0.0,-0.0,0.0,-0.0,0.0,-0.0,-193.25695,0.0,0.0,-0.0,0.0,0.0,0.0,-0.0,0.0,-0.0,-0.0,-0.0,0.0,-0.0,38.254867,0.0
2,31.12.2015,02:00,26671.532083,2015-12-31 02:00:00,-0.0,0.0,0.0,0.0,0.0,-0.0,-0.0,0.0,-0.0,0.0,-0.0,-0.0,-0.0,-0.0,-0.0,0.0,0.0,-0.0,-0.0,0.0,-0.0,0.0,-0.0,-193.25695,0.0,0.0,-0.0,0.0,0.0,0.0,-0.0,0.0,-0.0,-0.0,-0.0,0.0,-0.0,38.254867,0.0
3,31.12.2015,03:00,26246.622083,2015-12-31 03:00:00,-0.0,0.0,0.0,0.0,0.0,-0.0,-0.0,0.0,-0.0,0.0,-0.0,-0.0,-0.0,-0.0,-0.0,0.0,0.0,-0.0,-0.0,0.0,-0.0,0.0,-0.0,-193.25695,0.0,0.0,-0.0,0.0,0.0,0.0,-0.0,0.0,-0.0,-0.0,-0.0,0.0,-0.0,38.254867,0.0
4,31.12.2015,04:00,26026.542083,2015-12-31 04:00:00,-0.0,0.0,0.0,0.0,0.0,-0.0,-0.0,0.0,-0.0,0.0,-0.0,-0.0,-0.0,-0.0,-0.0,0.0,0.0,-0.0,-0.0,0.0,-0.0,0.0,-0.0,-193.25695,0.0,0.0,-0.0,0.0,0.0,0.0,-0.0,0.0,-0.0,-0.0,-0.0,0.0,-0.0,38.254867,0.0


In [41]:
# bruuuuuuuh
cons_holiday_binary=pd.read_csv("RealTimeConsumption_holiday.csv",encoding='latin-1')
cons_holiday_binary["Datetime"]=cons_holiday_binary["Date"]+" "+cons_holiday_binary["Hour"]
cons_holiday_binary["Datetime"]=pd.to_datetime(cons_holiday_binary["Datetime"],format="%d.%m.%Y %H:%M")
cons_holiday_binary.head()

,Date,Hour,Consumption,Datetime,is_annelergunu,is_arife,is_babalargunu,is_cocukbayrami,is_cumhuriyet,is_elections,is_examination,is_genclikbayrami,is_holiday,is_iscibayrami,is_kadinlargunu,is_kandil,is_kurbanbayrami,is_kurbanbayramilkgun,is_kurbanoncesigunler,is_onbestemmuz,is_outlierdays,is_ramazanbayrami,is_ramazanbayramilkgun,is_ramazanbayramisonrasi,is_ramazangunler,is_ramazanilkgun,is_ramazanoncesigunler,is_school_days,is_school_end,is_school_start,is_semester,is_semester_days,is_sevgililergunu,is_single_day_holidays,is_summerTimes,is_summer_break,is_uzunbayram,is_winterTimes,is_winter_break,is_winter_break_schools,is_yilbasi,is_yilbasioncesi,is_zaferbayrami
0,31.12.2015,00:00,29590.88,2015-12-31 00:00:00,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0
1,31.12.2015,01:00,27785.43,2015-12-31 01:00:00,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0
2,31.12.2015,02:00,26516.53,2015-12-31 02:00:00,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0
3,31.12.2015,03:00,26091.62,2015-12-31 03:00:00,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0
4,31.12.2015,04:00,25871.54,2015-12-31 04:00:00,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0


# Modelling

## CatBoost

In [42]:
cat_df_1=cons_holiday_binary.copy()
cat_df_1["Hour"]=cat_df_1["Hour"].apply(lambda x:x.split(":")[0]).astype(int)
cat_df_1["Day"]=cat_df_1["Date"].apply(lambda x:x.split(".")[0]).astype(int)
cat_df_1["Month"]=cat_df_1["Date"].apply(lambda x:x.split(".")[1]).astype(int)
cat_df_1["weekday"]=pd.to_datetime(cat_df_1["Date"]).apply(lambda x: x.weekday()).astype(int)
cat_df_1["Consumption_2_lag"]=cat_df_1["Consumption"].shift(24*2)
cat_df_1["Consumption_7_lag"]=cat_df_1["Consumption"].shift(24*7)
cat_df_1=cat_df_1.dropna()
cat_df_1.head()

,Date,Hour,Consumption,Datetime,is_annelergunu,is_arife,is_babalargunu,is_cocukbayrami,is_cumhuriyet,is_elections,is_examination,is_genclikbayrami,is_holiday,is_iscibayrami,is_kadinlargunu,is_kandil,is_kurbanbayrami,is_kurbanbayramilkgun,is_kurbanoncesigunler,is_onbestemmuz,is_outlierdays,is_ramazanbayrami,is_ramazanbayramilkgun,is_ramazanbayramisonrasi,is_ramazangunler,is_ramazanilkgun,is_ramazanoncesigunler,is_school_days,is_school_end,is_school_start,is_semester,is_semester_days,is_sevgililergunu,is_single_day_holidays,is_summerTimes,is_summer_break,is_uzunbayram,is_winterTimes,is_winter_break,is_winter_break_schools,is_yilbasi,is_yilbasioncesi,is_zaferbayrami,Day,Month,weekday,Consumption_2_lag,Consumption_7_lag
168,07.01.2016,0,28763.95,2016-01-07 00:00:00,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,7,1,4,29244.38,29590.88
169,07.01.2016,1,27284.84,2016-01-07 01:00:00,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,7,1,4,27637.23,27785.43
170,07.01.2016,2,26321.95,2016-01-07 02:00:00,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,7,1,4,26335.78,26516.53
171,07.01.2016,3,25748.49,2016-01-07 03:00:00,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,7,1,4,25870.23,26091.62
172,07.01.2016,4,25636.58,2016-01-07 04:00:00,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,7,1,4,25975.07,25871.54


In [0]:
try:
    from catboost import CatBoostRegressor
except ModuleNotFoundError:
    !pip install catboost
    from catboost import CatBoostRegressor

train_df=cat_df_1.loc[cat_df_1["Datetime"]<=pd.to_datetime("2019-03-03 23:00:00")]
valid_df=cat_df_1.loc[cat_df_1["Datetime"]>pd.to_datetime("2019-03-03 23:00:00")]

needed_colums=cat_df_1.columns.difference(["Datetime","Consumption","Date"])

model_cf = CatBoostRegressor(learning_rate=0.05,
                             num_boost_round=2000,
                             objective="MAE",
                             early_stopping_rounds=300,
                             #task_type="GPU",
                             random_state=1337,
                             depth=12,
                             #bootstrap_type="Poisson",
                             subsample=0.95,
                             max_bin=150)

cat_features_=[col for col in cat_df_1.columns if col.startswith("is")] # + ["Day","Month","weekday","Hour"]

eval_set=[(valid_df[needed_colums],valid_df["Consumption"])]

model_cf.fit(train_df[needed_colums],train_df["Consumption"],
          eval_set=eval_set,
          cat_features=[needed_colums.tolist().index(i) for i in cat_features_],verbose=2)

In [0]:
y_pred=model_cf.predict(valid_df[needed_colums])
y_true=valid_df["Consumption"].values
eval_function("Cat1",y_true,y_pred,valid_df[needed_colums.tolist()+["Datetime"]])

## CatBoost ccat

In [1]:
cat_df_1=cons_holiday_binary.copy()
cat_df_1["Hour"]=cat_df_1["Hour"].apply(lambda x:x.split(":")[0]).astype(int)
cat_df_1["Day"]=cat_df_1["Date"].apply(lambda x:x.split(".")[0]).astype(int)
cat_df_1["Month"]=cat_df_1["Date"].apply(lambda x:x.split(".")[1]).astype(int)
cat_df_1["weekday"]=pd.to_datetime(cat_df_1["Date"]).apply(lambda x: x.weekday()).astype(int)
cat_df_1["Consumption_2_lag"]=cat_df_1["Consumption"].shift(24*2)
cat_df_1["Consumption_7_lag"]=cat_df_1["Consumption"].shift(24*7)
cat_df_1=cat_df_1.dropna()
cat_df_1.head()

NameError: ignored

In [0]:
try:
    from catboost import CatBoostRegressor
except ModuleNotFoundError:
    !pip install catboost
    from catboost import CatBoostRegressor

train_df=cat_df_1.loc[cat_df_1["Datetime"]<=pd.to_datetime("2019-03-03 23:00:00")]
valid_df=cat_df_1.loc[cat_df_1["Datetime"]>pd.to_datetime("2019-03-03 23:00:00")]

needed_colums=cat_df_1.columns.difference(["Datetime","Consumption","Date"])

model_cf = CatBoostRegressor(learning_rate=0.05,
                             num_boost_round=2000,
                             objective="MAE",
                             early_stopping_rounds=300,
                             #task_type="GPU",
                             random_state=1337,
                             depth=12,
                             #bootstrap_type="Poisson",
                             subsample=0.95,
                             max_bin=150)

cat_features_=[col for col in cat_df_1.columns if col.startswith("is")] + ["Day","Month","weekday","Hour"]

eval_set=[(valid_df[needed_colums],valid_df["Consumption"])]

model_cf.fit(train_df[needed_colums],train_df["Consumption"],
          eval_set=eval_set,
          cat_features=[needed_colums.tolist().index(i) for i in cat_features_],verbose=2)

In [0]:
y_pred=model_cf.predict(valid_df[needed_colums])
y_true=valid_df["Consumption"].values
eval_function("Cat1",y_true,y_pred,valid_df[needed_colums.tolist()+["Datetime"]])